In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torchvision import datasets, transforms
import torch.nn as nn
import time
import datetime

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(8*8*8, 32)
    self.act3 = nn.ReLU()
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = self.pool1(self.act1(self.conv1(x)))
    out = self.pool2(self.act2(self.conv2(out)))
    out = out.view(-1, 8*8*8)
    out = self.act3(self.fc1(out))
    out = self.fc2(out)
    return out

In [ ]:
class NetB(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(2)
    self.conv3 = nn.Conv2d(8, 4, kernel_size=3, padding=1)
    self.act3 = nn.ReLU()
    self.pool3 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(4*4*4, 32)
    self.act4 = nn.ReLU()
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = self.pool1(self.act1(self.conv1(x)))
    out = self.pool2(self.act2(self.conv2(out)))
    out = self.pool3(self.act3(self.conv3(out)))
    out = out.view(-1, 4*4*4)
    out = self.act4(self.fc1(out))
    out = self.fc2(out)
    return out

In [ ]:
def training_loop(n_epochs, model, optimizer, loss_fn, train_loader):
  start_time = time.time()

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      outputs = model(imgs)
      loss = loss_fn(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_train += loss.item()

    print(f'{datetime.datetime.now()} Epoch {epoch}, Training loss {(loss_train / len(train_loader)):.4f}')

  end_time = time.time()
  elapsed_time = end_time - start_time
  print(f"Training completed in {elapsed_time:.2f} seconds.")


def validate(model, train_loader, val_loader):
  for name, loader in [('Training', train_loader), ('Validation', val_loader)]:
    correct = 0
    total = 0

    with torch.no_grad():
      for imgs, labels in loader:
        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

    print(f'{name} accuracy: {correct / total}')

In [ ]:
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True,
                             transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                  (0.2470, 0.2435, 0.2616))
                             ]))
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True,
                             transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.4942, 0.4851, 0.4504),
                                                  (0.2467, 0.2429, 0.2616))
                             ]))

100%|██████████| 170M/170M [00:04<00:00, 35.1MB/s]


Extracting ../data-unversioned/p1ch7/cifar-10-python.tar.gz to ../data-unversioned/p1ch7/
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=True)

model = Net()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 20,
    model = model,
    optimizer = optimizer,
    loss_fn = loss_fn,
    train_loader = train_loader
)

validate(model, train_loader, val_loader)

2024-12-05 00:11:06.144361 Epoch 1, Training loss 2.1012
2024-12-05 00:11:33.500231 Epoch 2, Training loss 1.6747
2024-12-05 00:12:00.572168 Epoch 3, Training loss 1.4655
2024-12-05 00:12:28.599565 Epoch 4, Training loss 1.3737
2024-12-05 00:12:55.393625 Epoch 5, Training loss 1.3186
2024-12-05 00:13:22.820662 Epoch 6, Training loss 1.2733
2024-12-05 00:13:49.947979 Epoch 7, Training loss 1.2334
2024-12-05 00:14:17.090650 Epoch 8, Training loss 1.2008
2024-12-05 00:14:43.941199 Epoch 9, Training loss 1.1732
2024-12-05 00:15:10.905414 Epoch 10, Training loss 1.1444
2024-12-05 00:15:37.446729 Epoch 11, Training loss 1.1244
2024-12-05 00:16:05.182844 Epoch 12, Training loss 1.1029
2024-12-05 00:16:31.867827 Epoch 13, Training loss 1.0818
2024-12-05 00:16:58.355076 Epoch 14, Training loss 1.0637
2024-12-05 00:17:25.303852 Epoch 15, Training loss 1.0474
2024-12-05 00:17:52.046049 Epoch 16, Training loss 1.0331
2024-12-05 00:18:19.794149 Epoch 17, Training loss 1.0194
2024-12-05 00:18:47.216

# Part B

In [ ]:
model = NetB()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 20,
    model = model,
    optimizer = optimizer,
    loss_fn = loss_fn,
    train_loader = train_loader
)

validate(model, train_loader, val_loader)

2024-12-05 00:20:34.939390 Epoch 1, Training loss 2.3031
2024-12-05 00:21:02.044014 Epoch 2, Training loss 2.2866
2024-12-05 00:21:29.231665 Epoch 3, Training loss 2.0317
2024-12-05 00:21:56.330517 Epoch 4, Training loss 1.7494
2024-12-05 00:22:24.130504 Epoch 5, Training loss 1.6074
2024-12-05 00:22:51.601569 Epoch 6, Training loss 1.5399
2024-12-05 00:23:20.038406 Epoch 7, Training loss 1.4970
2024-12-05 00:23:47.527428 Epoch 8, Training loss 1.4617
2024-12-05 00:24:14.957125 Epoch 9, Training loss 1.4298
2024-12-05 00:24:42.364585 Epoch 10, Training loss 1.4002
2024-12-05 00:25:10.154660 Epoch 11, Training loss 1.3742
2024-12-05 00:25:38.101195 Epoch 12, Training loss 1.3476
2024-12-05 00:26:05.673626 Epoch 13, Training loss 1.3254
2024-12-05 00:26:35.204947 Epoch 14, Training loss 1.3012
2024-12-05 00:27:03.875655 Epoch 15, Training loss 1.2809
2024-12-05 00:27:31.522847 Epoch 16, Training loss 1.2620
2024-12-05 00:27:58.856802 Epoch 17, Training loss 1.2465
2024-12-05 00:28:26.615